In [34]:
import pandas as pd 
import numpy as np 

from scipy.optimize import root 

In [5]:
grid_buses = pd.read_excel("grid_data.xlsx", sheet_name="busbars")
grid_lines = pd.read_excel("grid_data.xlsx", sheet_name="lines")
grid_loads = pd.read_excel("grid_data.xlsx", sheet_name="loads")
grid_gens  = pd.read_excel("grid_data.xlsx", sheet_name="gens")


In [104]:
# Find the base values 
S_base_mva = grid_gens["S_rated_mva"].sum()

# Make the y_bus: 
y_bus = np.zeros((grid_buses.shape[0], grid_buses.shape[0]), dtype=np.complex64)
for _, row in grid_lines.iterrows(): 
    v_base = row["v_nom_kv"]
    r = row["r_ohm_per_km"]
    x = row["x_ohm_per_km"] 
    c = row["c_uf_per_km"]
    i = row["from_bus_idx"]
    j = row["to_bus_idx"]
    z_base = v_base**2/S_base_mva
    z_line = (r + 1j*x)/z_base
    b_line_half = 2*np.pi*50*c*1e-6 / 2 * z_base #TODO: Find the frequency from somewhere else

    y_bus[i, j] = -(z_line**-1)
    y_bus[j, i] = -(z_line**-1)
    y_bus[i, i] += b_line_half
    y_bus[j, j] += b_line_half

for i, y_row in enumerate(y_bus): 
    y_bus[i, i] = - sum(y_row) + y_row[i] 


In [105]:
V_vals = np.array([1.0, 1.0])
delta_vals = np.array([0.0, 0.0])
P_vals = np.array([0, -10])/S_base_mva
Q_vals = np.array([0, -5])/S_base_mva

def get_pf_eqs(): 
    def pf_eq(X): 
        _delta = delta_vals.copy() 
        _v = V_vals.copy() 
        _delta[1] = X[0]
        _v[1] = X[1]
        v_vec = _v*np.cos(_delta) + _v*np.sin(_delta) * 1j 

        S_conj = v_vec.conj() * (y_bus @ v_vec)
        P_calc = S_conj.real 
        Q_calc = -S_conj.imag 
        root_diff = np.array([P_vals[1] - P_calc[1], Q_vals[1] - Q_calc[1]])
        return root_diff 
    return pf_eq 

In [106]:
X0 = np.array([0.0, 1.0])
func = get_pf_eqs()
sol = root(func, X0)
delta_vals[1] = sol.x[0]  
V_vals[1] = sol.x[1] 

v_vec = V_vals*np.cos(delta_vals) + V_vals*np.sin(delta_vals) * 1j 
S_conj = v_vec * (y_bus @ v_vec)
P_calc = S_conj.real 
Q_calc = -S_conj.imag 
sol

 message: The solution converged.
 success: True
  status: 1
     fun: [-3.488e-12 -1.892e-11]
       x: [-7.781e-02  9.304e-01]
    nfev: 9
    fjac: [[-9.365e-01  3.507e-01]
           [-3.507e-01 -9.365e-01]]
       r: [ 5.492e+00 -1.032e+00  5.015e+00]
     qtf: [-6.892e-10  3.886e-09]

In [107]:
print(f"delta_2 = {sol.x[0]*180/np.pi}, V_2 = {sol.x[1]}")

delta_2 = -4.457966848241377, V_2 = 0.9303508649967773


In [108]:
P_calc * S_base_mva

array([10.29838097, -9.1042424 ])

In [109]:
Q_calc * S_base_mva

array([ 6.1935239 , -6.48943529])

In [1]:
from grid_model import GridDataClass, GridModel
import numpy as np
from scipy.optimize import root 

In [2]:
grid_data = GridDataClass("grid_data.xlsx", f_nom=50)
grid_model = GridModel(grid_data) 

func = grid_model.setup_pf()

In [3]:
X0 = np.array([0.0, 1.0])
sol = root(func, X0)
print(f"delta_2 = {sol.x[0]*180/np.pi}, V_2 = {sol.x[1]}")

delta_2 = -4.038357723115159, V_2 = 0.8801433005619176


delta_2 = 68.22472610440269, V_2 = 1.3480853923910094


In [8]:
func(X0) 

array([  0., -10.,  -0.,  -5.])

In [9]:
grid_data.get_PQ_mask() 

(array([0, 1]), array([0, 1]))

In [11]:
a = np.array([0,1,2,3,4,5])
a[np.array([0,1,0,1,0,1])]

array([0, 1, 0, 1, 0, 1])